In [0]:
USE CATALOG claims360_dev;

-- ===== DENIAL RATE BY PAYER BY MONTH =====

CREATE OR REPLACE VIEW gold.v_monthly_denial_rate AS
SELECT
  date_trunc('month', submission_date) AS month,
  payer_name,
  COUNT(*) AS submitted_claims,
  SUM(has_denial_any) AS claims_with_denial,
  SUM(CASE WHEN current_balance <= 0 THEN 1 ELSE 0 END) AS paid_closed_claims,
  CAST(SUM(has_denial_any) AS DOUBLE) / COUNT(*) * 100 AS denial_rate_pct
FROM gold.fact_claim
GROUP BY 1, 2;


In [0]:
-- ===== OPEN AR BY PAYER =====

CREATE OR REPLACE VIEW gold.v_aging_buckets AS
SELECT
  payer_name,
  CASE
    WHEN current_balance <= 0 THEN 'Closed'
    WHEN DATEDIFF(current_date(), submission_date) <= 30  THEN '0–30'
    WHEN DATEDIFF(current_date(), submission_date) <= 60  THEN '31–60'
    WHEN DATEDIFF(current_date(), submission_date) <= 90  THEN '61–90'
    WHEN DATEDIFF(current_date(), submission_date) <= 120 THEN '91–120'
    ELSE '120+'
  END AS aging_bucket,
  SUM(current_balance) AS ar_amount,
  COUNT(*)             AS claim_count
FROM gold.fact_claim
GROUP BY 1, 2;

In [0]:
-- ===== TOP DENIAL CATEGORIES =====
CREATE OR REPLACE VIEW gold.v_top_denial_categories_events AS
SELECT
  p.payer_name,
  COALESCE(f.reason_category, 'Unmapped')          AS denial_category,
  COUNT(DISTINCT f.claim_id)                       AS claims_with_this_category,
  SUM(-1 * COALESCE(f.adjustment_amount, 0))       AS denial_amount
FROM gold.fact_denial_event f
LEFT JOIN silver.dim_payer p
  ON p.payer_id = f.payer_id
-- optional date slicing for dashboards:
-- WHERE f.event_ts >= DATEADD(day, -90, CURRENT_DATE())
GROUP BY p.payer_name, COALESCE(f.reason_category, 'Unmapped')
ORDER BY denial_amount DESC;

In [0]:
-- ===== FIRST-PASS YIELD =====
-- claims with at least one 835 payment event and no 277 reject/request and no 835 denial, over all submitted claims in period.

CREATE OR REPLACE VIEW gold.v_first_pass_yield AS
WITH base AS (
  SELECT
    DATE_FORMAT(c.submission_date, 'yyyy-MM') AS submission_month,
    c.payer_name,
    CASE WHEN c.submission_date <= date_sub(current_date(), 14) THEN 1 ELSE 0 END AS mature_flag,
    -- numerator flag: paid + no denial signals
    CASE
      WHEN c.last_835_event_ts IS NOT NULL
           AND COALESCE(c.has_denial_any, 0) = 0
      THEN 1 ELSE 0
    END AS is_fpy
  FROM gold.fact_claim c
)
SELECT
  submission_month,
  payer_name,
  COUNT(*) FILTER (WHERE mature_flag = 1)                                      AS submitted_claims,
  SUM(is_fpy) FILTER (WHERE mature_flag = 1)                                   AS fpy_claims,
  CAST(SUM(is_fpy) FILTER (WHERE mature_flag = 1) AS DOUBLE)
  / NULLIF(COUNT(*) FILTER (WHERE mature_flag = 1), 0)                          AS fpy_rate
FROM base
GROUP BY submission_month, payer_name;